First, load the 311 complaints dataset available here https://data.cityofnewyork.us/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9

In [ ]:
# 1. **Recurrent Neural Networks (RNNs) and its variants**:
#    - If you have a temporal sequence of complaints (i.e., time series data of complaints), then using LSTM (Long Short-Term Memory) or GRU (Gated Recurrent Unit) can be beneficial. These are designed to remember patterns over time, which can be useful for predicting events based on historical sequences.
# 2. **Convolutional Neural Networks (CNNs) for spatial data**:
#    - If the complaints can be represented as a spatial grid (like a heatmap over a city), CNNs can be a good choice. CNNs can identify spatial patterns that might be indicative of future complaints.
# 3. **Spatial-Temporal Models**:
#    - **ST-ResNet (Spatial-Temporal Residual Networks)**: This model captures both spatial and temporal patterns in data, making it particularly suitable for predicting city-wide crowd flows, and it could be adapted to predicting complaints.
#    - **Transformer-based models**: If you want to capture both spatial and temporal dependencies, you might consider using the Transformer architecture or its variants, which have proven effective in capturing complex relationships in data.
# 4. **Probabilistic Models**:
#    - **Gaussian Processes (GP)**: GPs can provide a probabilistic estimate of the next complaint location, taking into account uncertainty.
#    - **Hidden Markov Models (HMM)**: If there's a sequential aspect to the data, HMMs can be useful, although they might be less effective for very large datasets.
# 5. **Tree-based Models**:
#    - **XGBoost or LightGBM**: These gradient boosting algorithms can be very effective, especially if you engineer the right features, such as density of complaints in neighboring areas, recent trend in a particular area, etc.
# 6. **Neural Spatial-Temporal Models**:
#    - **Graph Neural Networks (GNNs)**: If the locations have a network structure (like interconnected nodes or regions), then GNNs can be effective in modeling the interdependencies.

In [7]:
CSV_PATH = './311.csv'
import subprocess
from tqdm import tqdm
import pandas as pd 
import os
from h3 import h3
import warnings
warnings.filterwarnings('ignore')


def file_len(fname):
    p = subprocess.Popen(['wc', '-l', fname], stdout=subprocess.PIPE, 
                                              stderr=subprocess.PIPE)
    result, err = p.communicate()
    if p.returncode != 0:
        raise IOError(err)
    return int(result.strip().split()[0])+1

n_rows = file_len(CSV_PATH)
print (f'Exact number of rows: {n_rows}')
df_tmp = pd.read_csv(CSV_PATH, nrows=5)
df_tmp.head()


Exact number of rows: 24895047


,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,...,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Latitude,Longitude,Location
0,41142518,12/09/2018 04:02:45 PM,12/10/2018 02:36:09 PM,DOT,Department of Transportation,Street Condition,Defective Hardware,Street,10314,211 NEAL DOW AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.618526,-74.135466,"(40.61852559304955, -74.1354659496114)"
1,41142519,12/09/2018 06:35:00 AM,12/09/2018 06:35:00 AM,DEP,Department of Environmental Protection,Water System,Possible Water Main Break (Use Comments) (WA1),NaN,10456,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.835601,-73.913793,"(40.83560095760538, -73.91379272762778)"
2,41142520,12/09/2018 08:23:09 AM,12/09/2018 08:23:09 AM,DOB,Department of Buildings,General Construction/Plumbing,Building Shaking/Vibrating/Structural Stability,NaN,10469,2761 BRONXWOOD AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.866606,-73.861541,"(40.86660642550429, -73.86154123573836)"
3,41142521,12/09/2018 01:38:03 PM,12/10/2018 12:16:46 AM,NYPD,New York City Police Department,Graffiti,Police Report Requested,Store/Commercial,11211,387 BROADWAY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.707272,-73.954561,"(40.70727200244776, -73.95456095910586)"
4,41142522,12/09/2018 08:44:00 AM,12/11/2018 12:00:00 PM,DSNY,A - Staten Island,Dirty Conditions,E3 Dirty Sidewalk,Sidewalk,10306,2704 HYLAN BOULEVARD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.566368,-74.113873,"(40.56636758759551, -74.11387270995246)"


In [13]:
types = {
    #'Unique Key': 'float32',
              'Complaint Type': 'category', 
              'Longitude': 'float32',
              'Latitude': 'float32',
#              'Incident Zip': 'int',
#              'Created Date': 'str',
#              'Closed Date': 'str',

        }
cols = list(types.keys())
chunksize = 5_000_000

df_list = [] # list to hold the batch dataframe

def process_date(df_chunk, key):
    df_chunk[key] = df_chunk[key].str.slice(0, 16)
    #df_chunk[key] = pd.to_datetime(df_chunk[key], utc=True, format='%Y-%m-%d %H:%M')
    
for df_chunk in tqdm(pd.read_csv(CSV_PATH, usecols=cols, dtype=types, chunksize=chunksize)):
    # Neat trick from https://www.kaggle.com/btyuhas/bayesian-optimization-with-xgboost
    # Using parse_dates would be much slower!
#    process_date(df_chunk, 'Closed Date')
#    process_date(df_chunk, 'Created Date')
    df_list.append(df_chunk) 

    
pdf = pd.concat(df_list)

# Delete the dataframe list to release memory
del df_list

# See what we have loaded
pdf.info()
#takes 1:30 minutes

5it [01:29, 17.92s/it]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24895045 entries, 0 to 24895044
Data columns (total 3 columns):
 #   Column          Dtype  
---  ------          -----  
 0   Complaint Type  object 
 1   Latitude        float32
 2   Longitude       float32
dtypes: float32(2), object(1)
memory usage: 379.9+ MB


In [14]:
pd.options.display.max_rows = 2000

counts = pdf['Complaint Type'].value_counts()
print(counts)



                               53404
HPD Literature Request                         52831
Illegal Fireworks                              52453
Dead/Dying Tree                                52312
Animal Abuse                                   52178
Housing - Low Income Senior                    51517
Curb Condition                                 49611
Indoor Air Quality                             49304
Vending                                        46600
Highway Condition                              45229
Emergency Response Team (ERT)                  44840
Traffic                                        44684
Noise - Park                                   42877
Water Conservation                             42085
Special Projects Inspection Team (SPIT)        41428
DHS Advantage - Tenant                         40930
Homeless Encampment                            38208
Electrical                                     38070
Electronics Waste Appointment                  32632
Food Pois

In [17]:
APERTURE_SIZE = 9
hex_col = 'hex'+str(APERTURE_SIZE)


pdf[hex_col] = 0
import json

print(1)
def binRow(df311):
    # find hexs containing the points
    df311[hex_col] = df311.apply(lambda x: h3.geo_to_h3(x.Latitude,x.Longitude,APERTURE_SIZE),1)

    # aggregate the points
    df311g = df311.groupby(hex_col).size()#.to_frame('cnt').reset_index()

    #find center of hex for visualization
#     df311g['lat'] = df311g[hex_col].apply(lambda x: h3.h3_to_geo(x)[0])
#     df311g['lng'] = df311g[hex_col].apply(lambda x: h3.h3_to_geo(x)[1])
    return (df311g)


def saveComplaint(complaint):
    print(complaint)
    data = pdf.loc[pdf['Complaint Type'] ==(complaint)]
    binned = binRow(data)
    complaint = complaint.replace(' ', '-').replace('/','-')
    hi = [(hex, count)  for hex,count in binned.iteritems()]
    with open('./data/'+complaint+'.json', 'w') as outfile: json.dump(hi, outfile)
    
stuff = ['Noise - Residential',                         
'HEAT/HOT WATER' ,                              
'Street Condition'   ,                           
'Illegal Parking ',                              
'Blocked Driveway'  ,                            
'Street Light Condition' ,                       
'HEATING'          ,                             
'PLUMBING'     ,                                 
'Water System'  ,                                
'Noise - Street/Sidewalk' ,                      
'GENERAL CONSTRUCTION'  ,                                                                
'UNSANITARY CONDITION']                          
for complaint in stuff:
    saveComplaint(complaint)



1
Noise - Residential
HEAT/HOT WATER
Street Condition
Illegal Parking 
Blocked Driveway
Street Light Condition
HEATING
PLUMBING
Water System
Noise - Street/Sidewalk
GENERAL CONSTRUCTION
UNSANITARY CONDITION


In [32]:
# data = pdf.loc[pdf['Complaint Type'] ==('Noise - Residential')]
# binned = binRow(data)
for hex, row in binned.iteritems():
    print(hex, row)

0 6593
872a10000ffffff 198
872a10002ffffff 216
872a10003ffffff 423
872a10006ffffff 2
872a10009ffffff 143
872a1000affffff 169
872a1000bffffff 1260
872a10010ffffff 25937
872a10011ffffff 6488
872a10012ffffff 54626
872a10013ffffff 32454
872a10014ffffff 14472
872a10015ffffff 2893
872a10016ffffff 18496
872a10018ffffff 5972
872a10019ffffff 1443
872a1001affffff 16976
872a1001bffffff 1492
872a1001cffffff 5469
872a1001dffffff 4347
872a1001effffff 25995
872a10033ffffff 67
872a10042ffffff 628
872a10046ffffff 23
872a10050ffffff 953
872a10051ffffff 3818
872a10052ffffff 1627
872a10053ffffff 3547
872a10054ffffff 508
872a10055ffffff 947
872a10056ffffff 1434
872a10058ffffff 1705
872a10059ffffff 156
872a1005affffff 4679
872a1005bffffff 2916
872a1005cffffff 327
872a1005effffff 6993
872a10073ffffff 9
872a10088ffffff 25697
872a10089ffffff 20769
872a1008bffffff 10205
872a1008cffffff 36002
872a1008dffffff 55584
872a100a1ffffff 30059
872a100a4ffffff 478
872a100a5ffffff 10321
872a100a8ffffff 71574
872a100a9ffff